In [1]:
import os

In [2]:
%pwd

'd:\\Text_Summarizer\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Text_Summarizer'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path


In [6]:
from src.constants import *
from src.utils import *

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAM_FILE_PATH):
        self.config=read_yaml(config_filepath)
        self.params=read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self):
        config=self.config.data_ingestion
        create_directories([config.root_dir])
        data_ingestion_config=DataIngestionConfig(config.root_dir,config.source_url,config.local_data_file,config.unzip_dir)
        return data_ingestion_config

In [10]:
import urllib.request as request
import zipfile
import gdown
from src.logger import logging

In [13]:
class DataIngestion:
    def __init__(self,config):
        self.config=config
    def download_file(self):
        dataset_url=self.config.source_url
        zip_file_path=self.config.local_data_file
        os.makedirs('artifacts/data_ingestion',exist_ok=True)
        logging.info("downloading dataset from {url_path} into file {zip_file_path}")
        file_id=dataset_url.split('/')[-2]
        prefix_url='https://drive.google.com/uc?/export=download&id='
        gdown.download(prefix_url+file_id,zip_file_path)
        logging.info("dataset downloaded successfully")
    def unzipfile(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        logging.info("unzipping the downloaded file")
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logging.info("unzipping of file successfull")

In [12]:
from src.exception import *

In [14]:
##pipeline
try:
    config=ConfigurationManager()
    data_ingestion_config=config.get_data_ingestion_config()
    data_ingestion=DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.unzipfile()
except Exception as e:
    logging.info("ERROR OCCURED IN THE PIPELINE")
    raise CustomException(e,sys)

Downloading...
From: https://drive.google.com/uc?/export=download&id=1Ov7i-MhptZkuR-_ZAMPSD5z47dXHaVXg
To: d:\Text_Summarizer\artifacts\data_ingestion\data.zip
100%|██████████| 4.24M/4.24M [00:02<00:00, 1.55MB/s]
